# 0.Introduction

This project is a playground to practice Machine Learning Linear Regression. The final objective is to predict the houses sale prices of the city of Ames, Iowa, United States from 2006 to 2010.

More info about the data:
- https://www.tandfonline.com/doi/abs/10.1080/10691898.2011.11889627
- https://s3.amazonaws.com/dq-content/307/data_description.txt

In [105]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, cross_val_score

In [106]:
data = pd.read_csv('AmesHousing.tsv', sep='\t')
data.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,...,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,...,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,...,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,...,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


The project is divided in main steps as the following:

- **Feature Engineering** using transform_features()
- **Feature Selection** using select_features()
- **Train and Test** using train_and_test()

In [107]:
def transform_features(df):
    cleanedDf = df.copy()
    cleanedDf['year_until_remod'] = cleanedDf['Year Remod/Add'] - cleanedDf['Year Built']
    dropedCols = ['Mo Sold', 'Yr Sold', 'Sale Type', 'Sale Condition', 'Order', 'PID', 'Year Remod/Add', 'Year Built', 'Garage Yr Blt']
    nullCount = data.isnull().sum()
    dropedCols += list(nullCount[nullCount > nbRecords*0.10].index)
    cleanedDf = cleanedDf.drop(dropedCols, axis=1)
    cleanedDf = cleanedDf.dropna() 
    return cleanedDf

def select_features(cleanedData):
    # selection based on correlation
    numericData = cleanedData.select_dtypes('number')
    corr = numericData.corr()['SalePrice'].abs()
    selectedFeatures = list(corr[corr > 0.4].index)
    numericData = numericData[selectedFeatures]
    #numeric category columns 
    numCatCols = ['Fireplaces', 'TotRms AbvGrd', 'Full Bath', 'Garage Cars', 'Overall Qual']
    numericData[numCatCols] = numericData[numCatCols].astype('category')
    numericData = pd.concat([numericData, pd.get_dummies(numericData[numCatCols])], axis=1)
    numericData = numericData.drop(numCatCols, axis=1)
    selectedData = numericData
    # non numeric features
    nonNumericData = cleanedData.select_dtypes('object')
    lowVarCatCols = []
    nbRecords = nonNumericData.shape[0]
    for col in nonNumericData.columns:
        for count in nonNumericData[col].value_counts():
            if(count > nbRecords*0.9):
                lowVarCatCols.append(col)
                break
    nonNumericData = nonNumericData.drop(lowVarCatCols, axis=1)
    nonNumericData = nonNumericData.astype('category')
    nonNumCols = nonNumericData.columns
    nonNumericData = pd.get_dummies(nonNumericData)
    selectedData = pd.concat([selectedData, nonNumericData], axis=1)
    return selectedData 

def train_and_test(df):
    cleanedData = transform_features(df)
    selectedData = select_features(cleanedData)
    model = LinearRegression()
    kf = KFold(5, shuffle=True)
    targetDf = selectedData[['SalePrice']]
    features = selectedData.columns.drop('SalePrice')
    featuresDf = selectedData[features]
    mses = cross_val_score(model, featuresDf, targetDf, scoring="neg_mean_squared_error", cv=kf)
    rmses = np.sqrt(-1*mses)
    return rmses.mean()

# 1. Feature Engineering 

Some guide lines:
- Remove features with too many missing values (more than 10% of the dataset)
- Transform text and categorical columns
- Normalize numerical columns
- Remove features that leak information about the target (sale price) like the year the sale happened

The cells below explore features in the dataset and the transformation needed is resumed in the function transform_features().

In [108]:
# a copy of the data so we can explore it without worrying about the original data
data2 = data.copy()

## Transform time features
The information of Year Built and Year Remod/Add hardly tells us about the sale price, but the difference between these 2 years could be an interesting feature showing how quick a house was remodeled.

In [109]:
data2['year_until_remod'] = data2['Year Remod/Add'] - data2['Year Built']

## Drop features
Some features need to be droped because they leak the information about the sale prices and they are not attributes of the houses.

We also drop columns with too many missing values.

In [110]:
dropedCols = ['Mo Sold', 'Yr Sold', 'Sale Type', 'Sale Condition', 'Order', 'PID', 'Garage Yr Blt']
dropedCols += ['Year Remod/Add', 'Year Built']

In [111]:
nullCount = data.isnull().sum()
nbRecords = data.shape[0]
nullCount[nullCount > nbRecords*0.10] # 10% of the number of records

Lot Frontage     490
Alley           2732
Fireplace Qu    1422
Pool QC         2917
Fence           2358
Misc Feature    2824
dtype: int64

In [112]:
dropedCols += list(nullCount[nullCount > nbRecords*0.10].index)

In [113]:
data2 = data2.drop(dropedCols, axis=1)

In [114]:
data2.head()

,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,...,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,year_until_remod
0,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,...,P,210,62,0,0,0,0,0,215000,0
1,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,Y,140,0,0,0,120,0,0,105000,0
2,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,...,Y,393,36,0,0,0,0,12500,172000,0
3,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,...,Y,0,0,0,0,0,0,0,244000,0
4,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,...,Y,212,34,0,0,0,0,0,189900,1


For the rest, we will see if we can drop rows having missing values.

In [115]:
print(data2.dropna().shape)
print(nbRecords)

(2678, 68)
2930


Yup, we still have 2678 records comparing to 2930 records in the original dataset. That's not bad. So I decided to drop all the rows having missing values.

In [116]:
data2 = data2.dropna()
data2.shape

(2678, 68)

## So...
All the process above will be rewritten in the function transform_features() which will return the cleaned data.

In [117]:
cleanedData = transform_features(data)
cleanedData.head()

,MS SubClass,MS Zoning,Lot Area,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,...,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,year_until_remod
0,20,RL,31770,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,...,P,210,62,0,0,0,0,0,215000,0
1,20,RH,11622,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,...,Y,140,0,0,0,120,0,0,105000,0
2,20,RL,14267,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,...,Y,393,36,0,0,0,0,12500,172000,0
3,20,RL,11160,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,...,Y,0,0,0,0,0,0,0,244000,0
4,60,RL,13830,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,...,Y,212,34,0,0,0,0,0,189900,1


# 2. Feature Selection

## Correlation between numeric columns and the target 'SalePrice'

In [118]:
numericData = cleanedData.select_dtypes('number')
numericData.head()

,MS SubClass,Lot Area,Overall Qual,Overall Cond,Mas Vnr Area,BsmtFin SF 1,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,1st Flr SF,...,Garage Area,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Misc Val,SalePrice,year_until_remod
0,20,31770,6,5,112.0,639.0,0.0,441.0,1080.0,1656,...,528.0,210,62,0,0,0,0,0,215000,0
1,20,11622,5,6,0.0,468.0,144.0,270.0,882.0,896,...,730.0,140,0,0,0,120,0,0,105000,0
2,20,14267,6,6,108.0,923.0,0.0,406.0,1329.0,1329,...,312.0,393,36,0,0,0,0,12500,172000,0
3,20,11160,7,5,0.0,1065.0,0.0,1045.0,2110.0,2110,...,522.0,0,0,0,0,0,0,0,244000,0
4,60,13830,5,5,0.0,791.0,0.0,137.0,928.0,928,...,482.0,212,34,0,0,0,0,0,189900,1


In [119]:
corr = numericData.corr()['SalePrice'].abs()
corr = corr.sort_values()
corr

BsmtFin SF 2        0.013117
Misc Val            0.014644
Low Qual Fin SF     0.028493
3Ssn Porch          0.030700
Bsmt Half Bath      0.046883
MS SubClass         0.073121
Pool Area           0.074451
Kitchen AbvGr       0.078667
Screen Porch        0.105232
Enclosed Porch      0.125185
Overall Cond        0.145874
Bedroom AbvGr       0.149441
Bsmt Unf SF         0.162408
year_until_remod    0.233285
Lot Area            0.261842
Bsmt Full Bath      0.263045
2nd Flr SF          0.265038
Half Bath           0.269085
Wood Deck SF        0.309485
Open Porch SF       0.317461
BsmtFin SF 1        0.412925
Fireplaces          0.452517
Mas Vnr Area        0.499652
TotRms AbvGrd       0.523363
Full Bath           0.555861
Total Bsmt SF       0.630305
1st Flr SF          0.630424
Garage Area         0.633467
Garage Cars         0.652448
Gr Liv Area         0.715867
Overall Qual        0.793249
SalePrice           1.000000
Name: SalePrice, dtype: float64

We can take only features which have the correlation to SalePrice greater than 0.4. (a parameter that need to be tested).

In [120]:
selectedFeatures = list(corr[corr > 0.4].index)
numericData = numericData[selectedFeatures]

print(selectedFeatures)

['BsmtFin SF 1', 'Fireplaces', 'Mas Vnr Area', 'TotRms AbvGrd', 'Full Bath', 'Total Bsmt SF', '1st Flr SF', 'Garage Area', 'Garage Cars', 'Gr Liv Area', 'Overall Qual', 'SalePrice']


In [121]:
numericData.columns

Index(['BsmtFin SF 1', 'Fireplaces', 'Mas Vnr Area', 'TotRms AbvGrd',
       'Full Bath', 'Total Bsmt SF', '1st Flr SF', 'Garage Area',
       'Garage Cars', 'Gr Liv Area', 'Overall Qual', 'SalePrice'],
      dtype='object')

## Look for category features hiding in numeric columns

In [122]:
for col in numericData.columns:
    print(col + ': ', len(numericData[col].unique()), " unique values")

BsmtFin SF 1:  974  unique values
Fireplaces:  5  unique values
Mas Vnr Area:  443  unique values
TotRms AbvGrd:  12  unique values
Full Bath:  5  unique values
Total Bsmt SF:  1031  unique values
1st Flr SF:  1047  unique values
Garage Area:  597  unique values
Garage Cars:  5  unique values
Gr Liv Area:  1236  unique values
Overall Qual:  9  unique values
SalePrice:  959  unique values


We notice some columns that could be in type category: Bsmt Full Bath, Half Bath, Fireplaces, TotRms AbvGrd, Full Bath, Garage Cars, Overall Qual

In [123]:
numCatCols = ['Fireplaces', 'TotRms AbvGrd', 'Full Bath', 'Garage Cars', 'Overall Qual']
numericData[numCatCols] = numericData[numCatCols].astype('category')
numericData = pd.concat([numericData, pd.get_dummies(numericData[numCatCols])], axis=1)
numericData = numericData.drop(numCatCols, axis=1)
numericData.head()

,BsmtFin SF 1,Mas Vnr Area,Total Bsmt SF,1st Flr SF,Garage Area,Gr Liv Area,SalePrice,Fireplaces_0,Fireplaces_1,Fireplaces_2,...,Garage Cars_5.0,Overall Qual_2,Overall Qual_3,Overall Qual_4,Overall Qual_5,Overall Qual_6,Overall Qual_7,Overall Qual_8,Overall Qual_9,Overall Qual_10
0,639.0,112.0,1080.0,1656,528.0,1656,215000,0,0,1,...,0,0,0,0,0,1,0,0,0,0
1,468.0,0.0,882.0,896,730.0,896,105000,1,0,0,...,0,0,0,0,1,0,0,0,0,0
2,923.0,108.0,1329.0,1329,312.0,1329,172000,1,0,0,...,0,0,0,0,0,1,0,0,0,0
3,1065.0,0.0,2110.0,2110,522.0,2110,244000,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,791.0,0.0,928.0,928,482.0,1629,189900,0,1,0,...,0,0,0,0,1,0,0,0,0,0


## Handle non-numeric columns

In [124]:
nonNumericData = cleanedData.select_dtypes('object')
nonNumericData.head()

,MS Zoning,Street,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,...,Heating QC,Central Air,Electrical,Kitchen Qual,Functional,Garage Type,Garage Finish,Garage Qual,Garage Cond,Paved Drive
0,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,...,Fa,Y,SBrkr,TA,Typ,Attchd,Fin,TA,TA,P
1,RH,Pave,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,...,TA,Y,SBrkr,TA,Typ,Attchd,Unf,TA,TA,Y
2,RL,Pave,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,...,TA,Y,SBrkr,Gd,Typ,Attchd,Unf,TA,TA,Y
3,RL,Pave,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,...,Ex,Y,SBrkr,Ex,Typ,Attchd,Fin,TA,TA,Y
4,RL,Pave,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,...,Gd,Y,SBrkr,TA,Typ,Attchd,Fin,TA,TA,Y


In [125]:
for col in nonNumericData.columns:
    print(col + ': ', len(nonNumericData[col].unique()), " unique values")

MS Zoning:  6  unique values
Street:  2  unique values
Lot Shape:  4  unique values
Land Contour:  4  unique values
Utilities:  3  unique values
Lot Config:  5  unique values
Land Slope:  3  unique values
Neighborhood:  28  unique values
Condition 1:  9  unique values
Condition 2:  8  unique values
Bldg Type:  5  unique values
House Style:  8  unique values
Roof Style:  6  unique values
Roof Matl:  8  unique values
Exterior 1st:  14  unique values
Exterior 2nd:  16  unique values
Mas Vnr Type:  5  unique values
Exter Qual:  4  unique values
Exter Cond:  5  unique values
Foundation:  5  unique values
Bsmt Qual:  5  unique values
Bsmt Cond:  5  unique values
Bsmt Exposure:  4  unique values
BsmtFin Type 1:  6  unique values
BsmtFin Type 2:  6  unique values
Heating:  4  unique values
Heating QC:  5  unique values
Central Air:  2  unique values
Electrical:  5  unique values
Kitchen Qual:  5  unique values
Functional:  8  unique values
Garage Type:  6  unique values
Garage Finish:  3  uniq

These non-numeric features have a small number of unique values, so we can handle these features as type 'category'. But which categorical columns have a few unique values but more than 90% of the values in the column belong to a specific category? This would be similar to a low variance numerical feature (no variability in the data for the model to capture).

## Find low-variance category columns

We will remove any columns that have a few unique values but more than 90% of the values in the column belong to a specific category.

In [126]:
lowVarCatCols = []
nbRecords = nonNumericData.shape[0]
for col in nonNumericData.columns:
    for count in nonNumericData[col].value_counts():
        if(count > nbRecords*0.9):
            lowVarCatCols.append(col)
            break
nonNumericData = nonNumericData.drop(lowVarCatCols, axis=1)
lowVarCatCols

['Street',
 'Utilities',
 'Land Slope',
 'Condition 2',
 'Roof Matl',
 'Bsmt Cond',
 'Heating',
 'Central Air',
 'Electrical',
 'Functional',
 'Garage Qual',
 'Garage Cond',
 'Paved Drive']

## The final nonNumericData

In [127]:
nonNumericData = nonNumericData.astype('category')
nonNumCols = nonNumericData.columns
nonNumericData = pd.get_dummies(nonNumericData)

In [128]:
nonNumericData.head()

,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Lot Shape_IR1,Lot Shape_IR2,Lot Shape_IR3,Lot Shape_Reg,...,Kitchen Qual_TA,Garage Type_2Types,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Finish_Fin,Garage Finish_RFn,Garage Finish_Unf
0,0,0,0,0,1,0,1,0,0,0,...,1,0,1,0,0,0,0,1,0,0
1,0,0,0,1,0,0,0,0,0,1,...,1,0,1,0,0,0,0,0,0,1
2,0,0,0,0,1,0,1,0,0,0,...,0,0,1,0,0,0,0,0,0,1
3,0,0,0,0,1,0,0,0,0,1,...,0,0,1,0,0,0,0,1,0,0
4,0,0,0,0,1,0,1,0,0,0,...,1,0,1,0,0,0,0,1,0,0


## Final cleaned, selected data

In [129]:
selectedData = pd.concat([nonNumericData, numericData], axis=1)

In [130]:
selectedData.head()

,MS Zoning_C (all),MS Zoning_FV,MS Zoning_I (all),MS Zoning_RH,MS Zoning_RL,MS Zoning_RM,Lot Shape_IR1,Lot Shape_IR2,Lot Shape_IR3,Lot Shape_Reg,...,Garage Cars_5.0,Overall Qual_2,Overall Qual_3,Overall Qual_4,Overall Qual_5,Overall Qual_6,Overall Qual_7,Overall Qual_8,Overall Qual_9,Overall Qual_10
0,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,0,0,1,0,0,0,0,0,1,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,0
4,0,0,0,0,1,0,1,0,0,0,...,0,0,0,0,1,0,0,0,0,0


The function select_features() would return the same DataFrame as above from cleaned data.

In [131]:
select_features(cleanedData).head()

,Mas Vnr Area,BsmtFin SF 1,Total Bsmt SF,1st Flr SF,Gr Liv Area,Garage Area,SalePrice,Fireplaces_0,Fireplaces_1,Fireplaces_2,...,Kitchen Qual_TA,Garage Type_2Types,Garage Type_Attchd,Garage Type_Basment,Garage Type_BuiltIn,Garage Type_CarPort,Garage Type_Detchd,Garage Finish_Fin,Garage Finish_RFn,Garage Finish_Unf
0,112.0,639.0,1080.0,1656,1656,528.0,215000,0,0,1,...,1,0,1,0,0,0,0,1,0,0
1,0.0,468.0,882.0,896,896,730.0,105000,1,0,0,...,1,0,1,0,0,0,0,0,0,1
2,108.0,923.0,1329.0,1329,1329,312.0,172000,1,0,0,...,0,0,1,0,0,0,0,0,0,1
3,0.0,1065.0,2110.0,2110,2110,522.0,244000,0,0,1,...,0,0,1,0,0,0,0,1,0,0
4,0.0,791.0,928.0,928,1629,482.0,189900,0,1,0,...,1,0,1,0,0,0,0,1,0,0


# 3. Train and Test

This is the last step of the project, where we train and test the Linear Regression model using K-Folds Cross-Validation.

In [134]:
train_and_test(data)

29023.86175123178

In [135]:
data.SalePrice.describe()

count      2930.000000
mean     180796.060068
std       79886.692357
min       12789.000000
25%      129500.000000
50%      160000.000000
75%      213500.000000
max      755000.000000
Name: SalePrice, dtype: float64

# 4.Conclusion

Our model has error about 29000 USD for each prediction of House Sale Price. That's not bad comparing to House Sale Prices mean of 180 000 USD and the standard deviation of its distribution about 80 000 USD.

Here is a [Kaggle forum](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/kernels) where you can find other notebooks on the same dataset of this project.